<h1>Explanation of the class Pietro_Wavelet_Transforms</h1>
<div class="text">
    The purpose of this class is to calculate the PS, S1, S2, PS3d and the $\ell^1$ and $\ell^2$ summaries.
    <br>
    For the initialization of this class, the brightness_temp of the coeval cube is not needed  
    For now there are two versions of this class:
    <ul>
        <li>
            Pietro_Wavelet_Transforms_1: this one doesn't have the normalization of S1 and S2. 
            <br>
            It is used when running simulations if we want to save the statistics PS, S1, S2 and PS3d (inside psbi.create_batched_data_0).
        </li>
        <li>
            Pietro_Wavelet_Transforms_3: this one doesn't have the normalization of S1 and S2.
            <br>
            This version has one additional function which is load_sim(). When loading the simulation the class immediately calculates PS, S1, S2 and PS3d and normalizes S1 and S2.
            <br>
            It is used when running simulations if we want to save the statistics $\ell^1$ and $\ell^2$ summaries (inside psbi.create_batched_data_2).
        </li>
    </ul>
</div>
<h3>Explanation of the four statistics (arXiv:2311.00036v2)</h3>
<ul>
    <li>
        $PS = \int_{\mathbb{R}^2}^{} |I(x) * \Psi_\lambda (x)|^2 \,dx $
    </li>
    <li>
        $S_1(\lambda) = \int_{\mathbb{R}^2}^{} |I(x) * \Psi_\lambda (x)| \,dx $
    </li>
    <li>
        $S_2(\lambda_1, \lambda_2) = \int_{\mathbb{R}^2}^{} |I(x) * \Psi_{\lambda_1} (x)| * \Psi_{\lambda_2} (x) \,dx $, with $\lambda_2 > \lambda_1$
    </li>
    <li>
        PS3d = $PS = \int_{\mathbb{R}^3}^{} |I(x) * \Psi_\lambda (x)|^2 \,dx $
    </li>
</ul>

In [1]:
import os
import sys
import numpy as np
sys.path.insert(1, os.path.abspath('../')) # Note that this line is useless with a regular pip installation of PyWST.
import pietrosbi as psbi

/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast/_cfg.py:57: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast/_cfg.py:41: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")


<h2>Creation of a coeval cube as example</h2>
<div class="text">
    Look the the first tutorial to understand the follwoing code
</div>

In [2]:
# All the paramters for the coeval cube (as explained in turotial 1)
z = 9 
n_pixels = 32 
dim = 300 # Mpc
astro_params_dict = {"HII_EFF_FACTOR": 30, "ION_Tvir_MIN": 4.7} 
seed = 1

# Creation of the coeval cube
coeval = psbi.create_coeval(z = z, 
                           n_pixels = n_pixels, 
                           dim = dim, 
                           astro_params_dict = astro_params_dict, 
                           seed = seed)

# Remove files from the cache
psbi.remove_21cmfast_cache()

/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast/_utils.py:400: UserWarning: The following parameters to FlagOptions are not supported: ['USE_VELS_AUX']
  warnings.warn(
/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast/_utils.py:815: UserWarning: Trying to remove array that isn't yet created: hires_vx
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast/_utils.py:815: UserWarning: Trying to remove array that isn't yet created: hires_vy
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast/_utils.py:815: UserWarning: Trying to remove array that isn't yet created: hires_vz
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast

<h2>Pietro_Wavelet_Transforms_1</h2>

In [3]:
# NUmber of bins for the statistics
bins = 6

# True if you want the l1 summary, otherwise False
l1 = True

# True if you want the l2 summary, otherwise False
l2 = True

# Type wavelet for the line of sight decomposition
wavelet_type = 'morl'

# Initialization of the object Pietro_Wavelet_Transforms_1
My_WT_1 = psbi.Pietro_Wavelet_Transforms_1(box_size=dim, n_pixels=n_pixels, bins=bins, l1=l1, l2=l2, wavelet_type = wavelet_type)

# To calculate the statistics PS, S1, S2 and PS3d of the coeval cube
# Note: the first three require a for loop over all the slices of the cube, since they are 2-dimensional statistics
data_S1_1 = np.array([My_WT_1.S1(coeval.brightness_temp[i], plot_fig = False) for i in range(len(coeval.brightness_temp))])
data_PS_1 = np.array([My_WT_1.PS(coeval.brightness_temp[i], plot_fig = False) for i in range(len(coeval.brightness_temp))])
data_S2_1 = np.array([My_WT_1.S2(coeval.brightness_temp[i], plot_fig = False) for i in range(len(coeval.brightness_temp))])
data_PS3d_1 = My_WT_1.PS(coeval.brightness_temp)

In [4]:
# S1 and S2 are not normalized.
# To normalize them you can use the following function taken from import_data_S1_S2_PS_PS3d.py
# If you run the multiple simulations and save the statistics in .npy files as explained in the 
# next tutorial, you don't need to do this normalization since it will be handled automatically 
# when importing the data (also explained in the next tutorial)
def norm_obs_data(data_obs_PS, data_obs_S1, data_obs_S2):
    data_obs_S1_rev = data_obs_S1[:,::-1]
    data_obs_S1_rev = data_obs_S1_rev[:,:,np.newaxis] 
    data_obs_S2_new = data_obs_S2 / data_obs_S1_rev 
    data_obs_S1_new = data_obs_S1 / np.sqrt(data_obs_PS)
    
    data_obs_S1_new[np.isnan(data_obs_S1_new)] = 0
    data_obs_S2_new[np.isnan(data_obs_S2_new)] = 0

    return data_obs_S1_new, data_obs_S2_new


data_S1_1, data_S2_1 = norm_obs_data(data_PS_1, data_S1_1, data_S2_1)

<h2>Pietro_Wavelet_Transforms_3</h2>

In [5]:
# NUmber of bins for the statistics
bins = 6

# True if you want the l1 summary, otherwise False
l1 = True

# True if you want the l2 summary, otherwise False
l2 = True

# Type wavelet for the line of sight decomposition
wavelet_type = 'morl'

# Initialization of the object Pietro_Wavelet_Transforms_1
My_WT_3 = psbi.Pietro_Wavelet_Transforms_3(box_size=dim, n_pixels=n_pixels, bins=bins, l1=l1, l2=l2, wavelet_type = wavelet_type)

# Loading the brightness temperature of the coeval cube.
# This function will immediately calculate PS, S1, S2 and PS3d and normalize S1 and S2
My_WT_3.load_sim(coeval.brightness_temp)

# The statistics PS, S1, S2 and PS3d are now attributes of this object
# and can be called in this way:
data_S1_3 = My_WT_3.S1
data_PS_3 = My_WT_3.PS
data_S2_3 = My_WT_3.S2
data_PS3d_3 = My_WT_3.PS3d

In [20]:
# Checking that no matter what version of the class we use we get the same arrays
print(np.all(np.round(data_PS_1,10) == np.round(data_PS_3,10)))
print(np.all(np.round(data_S1_1,10) == np.round(data_S1_3,10)))
print(np.all(np.round(data_S2_1,10) == np.round(data_S2_3,10)))
print(np.all(np.round(data_PS3d_1,10) == np.round(data_PS3d_3,10)))

True
True
True
True
